In [ ]:
import sys
sys.path.append('..')
from config_figures import *

In [ ]:
ecoregions_species_sampled = pd.read_csv(sample_ecoregion_file)
ecoregions_species_sampled['ECO_ID'] = ecoregions_species_sampled.apply(lambda x: x.site.split('_')[1], axis=1).astype('int')
ecoregions_species_sampled['current_or_future'] = ecoregions_species_sampled.apply(lambda x: x.site.split('_')[2], axis=1)
ecoregions_species_sampled = ecoregions_species_sampled.drop(columns=['site']).set_index(['ECO_ID','current_or_future'])
ecoregions_species_sampled

In [ ]:
sdms_area_lat_elev = pd.read_csv(sdms_area_lat_elev_file, index_col=['species','climate']).drop(columns=['system:index','.geo'])
sdms_area_lat_elev = sdms_area_lat_elev[sdms_area_lat_elev['min_tree_cover'] == 0]
sdms_area_lat_elev

In [ ]:
def lat_elev_shift(row):
    species_df = sdms_area_lat_elev.loc[row.species]
    delta = species_df.loc['2071_2100_ssp585'] - species_df.loc['1981_2010']
    row['abs_lat_shift'] = abs(delta['median_lat'])
    row['elev_shift'] = delta['median_elev']
    return row

lat_elev_shift_df = pd.DataFrame({
    'species': sdms_area_lat_elev.index.get_level_values('species').copy().drop_duplicates()
}).apply(lat_elev_shift, axis=1)
lat_elev_shift_df

In [ ]:
def n_lost_gained(row):
    eco_species = ecoregions_species_sampled.loc[row.ECO_ID]

    delta_species = (eco_species.loc['current'] - eco_species.loc['future']).to_list()
    row['n_current'] = eco_species.loc['current'].to_list().count(1)
    row['n_future'] = eco_species.loc['future'].to_list().count(1)
    row['n_lost'] = delta_species.count(1)
    row['n_gained'] = delta_species.count(-1)

    species_current_future = eco_species.columns[eco_species.sum() == 2].to_list()
    species_lat_elev_shift = lat_elev_shift_df[lat_elev_shift_df['species'].isin(species_current_future)]
    row['median_abs_lat_shift'] = species_lat_elev_shift['abs_lat_shift'].median()
    row['median_elev_shift'] = species_lat_elev_shift['elev_shift'].median()

    return row

eco_df = pd.DataFrame({
    'ECO_ID': ecoregions_species_sampled.index.get_level_values('ECO_ID').copy().drop_duplicates()
}).apply(n_lost_gained, axis=1)
eco_df['frac_gained'] = (eco_df['n_gained'] / eco_df['n_current']) * 100
eco_df['frac_lost'] = (eco_df['n_lost'] / eco_df['n_current']) * 100
eco_df

In [ ]:
nmds_eucl_dist = pd.read_csv('nmds_current_future_eucl_dist.csv')[['ECO_ID', 'eucl_dist', 'eucl_dist_scaled']].rename(
    columns={'eucl_dist': 'NMDS_eucl_dist', 'eucl_dist_scaled': 'NMDS_eucl_dist_scaled'}
)
evopca_eucl_dist = pd.read_csv('evoPCA_current_future_eucl_dist.csv')[['ECO_ID', 'eucl_dist', 'eucl_dist_scaled']].rename(
    columns={'eucl_dist': 'evoPCA_eucl_dist', 'eucl_dist_scaled': 'evoPCA_eucl_dist_scaled'}
)
nmds_evopca_df = pd.merge(nmds_eucl_dist, evopca_eucl_dist)

eco_df = pd.merge(eco_df, nmds_evopca_df)
eco_df 

In [ ]:
forest_ecoregions = ecoregions.filter(ee.Filter.stringContains('BIOME_NAME', 'Forest')).getInfo()['features']

In [ ]:
props = ['BIOME_NAME','ECO_ID']
eco_df = pd.merge(eco_df, pd.DataFrame([[item['properties'][p] for p in props] for item in forest_ecoregions], columns=props))

In [ ]:
eco_df.to_csv(climate_change_ecoregion_file, index=False)

In [ ]:
eco_df = pd.read_csv(climate_change_ecoregion_file)
eco_df

In [ ]:
biome_names = {
    'Boreal Forests/Taiga': 'Boreal Forests',
    'Mediterranean Forests, Woodlands & Scrub': 'Mediterranean Forests',
    'Temperate Broadleaf & Mixed Forests': 'Temperate Broadleaf Forests', 
    'Temperate Conifer Forests': 'Temperate Conifer Forests',
    'Tropical & Subtropical Coniferous Forests': 'Tropical Coniferous Forests',
    'Tropical & Subtropical Dry Broadleaf Forests': 'Tropical Dry Broadleaf Forests',
    'Tropical & Subtropical Moist Broadleaf Forests': 'Tropical Moist Broadleaf Forests'
}
eco_df['BIOME_NAME'] = eco_df['BIOME_NAME'].apply(lambda b: biome_names[b])

In [ ]:
eco_df.groupby('BIOME_NAME').ECO_ID.count()

In [ ]:
df_melted = pd.melt(eco_df, id_vars=['BIOME_NAME','ECO_ID'], value_vars=[
    'frac_lost', 'frac_gained', 'median_abs_lat_shift', 'median_elev_shift', 
    'NMDS_eucl_dist_scaled', 'evoPCA_eucl_dist_scaled'
]) 
df_melted

In [ ]:
sns.set(style='whitegrid')
biome_order = [
    'Tropical Moist Broadleaf Forests', 
    'Tropical Coniferous Forests',
    'Tropical Dry Broadleaf Forests', 
    'Boreal Forests', 
    'Temperate Broadleaf Forests', 
    'Temperate Conifer Forests', 
    'Mediterranean Forests'
]
palette=['#F0E442', '#56B4E9', '#E69F00', '#009E73', '#CC79A7', '#0072B2', '#D55E00']

g = sns.catplot(
    data=df_melted, y='BIOME_NAME', x='value', col='variable', kind='box', 
    sharex=False, col_wrap=3, palette=palette, order=biome_order,
    col_order=['median_abs_lat_shift','frac_gained','evoPCA_eucl_dist_scaled',
               'median_elev_shift','frac_lost','NMDS_eucl_dist_scaled'],
    height=2, aspect=1.25, flierprops={'marker':'o', 'markersize':1}, linewidth=0.5
)
g.set_ylabels("")

# g.set_xticklabels(rotation=90)
for ax, lab in zip(g.axes, [
    'Absolute latitude shift (°)', 'Fraction gained species (%)', 'Change in phylogenetic composition',
    'Elevation shift (m)', 'Fraction lost species (%)', 'Change in taxonomic composition'
]):
    ax.set_xlabel(lab, fontsize=7)
    ax.tick_params(labelsize=7)
    ax.set_title("")
    ax.spines['bottom'].set_linewidth(0.5)
    ax.spines['left'].set_linewidth(0.5)

g.axes[0].set_xlim(0,7)

plt.savefig(figuredir + 'climate_change_boxplots.pdf', bbox_inches='tight')

# Statistics

In [ ]:
from sklearn.preprocessing import scale
from statsmodels.multivariate.manova import MANOVA

In [ ]:
manova_result = MANOVA.from_formula(
    'median_abs_lat_shift + median_elev_shift + frac_lost + frac_gained + NMDS_eucl_dist_scaled + evoPCA_eucl_dist_scaled ~ BIOME_NAME', 
    eco_df)
print(manova_result.mv_test())

In [ ]:
# Coefficient of variation 
vars = [
    'frac_lost', 'frac_gained', 'median_abs_lat_shift', 'median_elev_shift', 'NMDS_eucl_dist_scaled', 'evoPCA_eucl_dist_scaled'
]
eco_df[vars].std() / eco_df[vars].mean()

## Absolute latitude shift

In [ ]:
eco_df.groupby('BIOME_NAME').median_abs_lat_shift.quantile([0.25,0.5,0.75]) 

In [ ]:
# absolute latitude shift in ecoregions in tropical forest biomes
tropical_abs_lat_shift = eco_df[eco_df.BIOME_NAME.str.contains('Tropical')].median_abs_lat_shift
tropical_abs_lat_shift.quantile([0.25, 0.5,0.75]) 

## Elevation shift

In [ ]:
eco_df.groupby('BIOME_NAME').median_elev_shift.median()#quantile([0.25,0.5,0.75])

In [ ]:
eco_df[(eco_df.median_elev_shift < 5) & (eco_df.median_elev_shift > -5)].shape[0] / eco_df.shape[0]

## Fraction of species lost and gained

In [ ]:
eco_df.groupby('BIOME_NAME').frac_lost.agg(['median','min','max'])